In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
import os
from deepcell.applications import Mesmer
from deepcell.utils.plot_utils import make_outline_overlay
from deepcell.utils.tracking_utils import trks_stats, load_trks
from deepcell.utils.plot_utils import create_rgb_image
import skimage.io as io
import skimage.segmentation

## Nuclear Segmentation 

In [2]:
def Mesmer_segmentor(in_dir, save_dir_seg):
    '''
    Produces label masks as nuclear segmentation output of Mesmer segementation algorithm

    Parameters
    in_dir: str
        the path to the directory containing the grayscaled DAPI channel images of various fields. All images should be in .tiff format
    save_dir_seg: str
        the path to the directory where the segmented images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''
    
    # iterating over all files in the input directory
    for file in os.listdir(in_dir):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # reads the grayscaled DAPI channel of each field as an array
            DAPI_gray = np.array([io.imread(os.path.join(in_dir, file))])
            # first and third channels should be zero for Mesmer to perform nuclear segmentation
            DAPI_gray[:, :, : , 0] = 0
            DAPI_gray[:, :, : , 2] = 0
            # Creates an "rgb image" where green channel is nucleus and the blue channel is cytoplasm (zeros over
            # here since performing only nuclear segmentation)
            rgb_images = create_rgb_image(DAPI_gray[:, :, :, 1:], channel_colors=['green', 'blue'])
            # predicting nuclei using the Mesmer model
            app = Mesmer()
            segmentation_predictions = app.predict(rgb_images[:, :, :, 1:], image_mpp=0.5, compartment='nuclear')
            # extracting nuclei label array
            label_mask = segmentation_predictions[0, :, :, 0]
            io.imsave(os.path.join(save_dir_seg, file[:file.find('.')] + '.tiff'), label_mask)
            
        else:
            pass
        
    return None

## Post Processing 
Post-processing of predicted label masks into binary masks for evaluation. The overall effect of this step is to separate touching nuclei by adding a 2-pixel gap (one-pixel erosion on each nucleus). Next, binary masks (both segmented and ground truth data) can be used to compute the F1-scores by utilizing connected component analysis. 

In [3]:
def postprocess(save_dir_seg, save_dir_bin):
    '''
    Produces binary masks for evaluation from the labelled mask output from the Mesmer algorithm

    Parameters
    save_dir_seg: str
        the path to the directory containing the label masks output by the Mesmer algorithm with the Mesmer_segmentor function. All images should
        be in .tiff format
    save_dir_bin: str
        the path to the directory where the binary images will be saved as masks. Each tiff image in the input directory will
        have a corresponding mask in the directory provided by the user
    '''

    # iterating over all files in the input directory
    for file in os.listdir(save_dir_seg):
        # only processes tiff images
        if file[-5:]=='.tiff' or file[-4:]=='.tif':
            # Finding the pixels that are touching between any two nuclei
            label_mask = io.imread(os.path.join(save_dir_seg, file))
            boundary_bool = skimage.segmentation.find_boundaries(label_mask, connectivity=label_mask.ndim,
                                                                 mode='outer', background=0)
            # Converting these pixels to the background value in the label array
            label_mask[boundary_bool] = 0
            # Converting the label array into a binary mask of foreground (255) and background (0)
            nuclei_mask_final = np.zeros((label_mask.shape[0], label_mask.shape[1]))
            nuclei_mask_final[label_mask != 0] = 255
            nuclei_mask_final = np.uint8(nuclei_mask_final)
            
            # saving the binary mask in the save directory
            io.imsave(os.path.join(save_dir_bin, file[:file.find('.')] + '.tiff'), nuclei_mask_final)
        else:
            pass
        
    return None

In [4]:
# Replace the strings for in_dir, save_dir_seg and save_dir_bin to paths on the local machine
in_dir = r'C:\Users\mrl\Desktop\nuclear_seg_github\imgs\DAPI_grayscale'
save_dir_seg = r'C:\Users\mrl\Desktop\nuclear_seg_github\imgs\Mesmer_label_masks'
save_dir_bin = r'C:\Users\mrl\Desktop\nuclear_seg_github\imgs\Mesmer_binary_masks'
# To create binary masks using Mesmer segmentation algorithm
Mesmer_segmentor(in_dir, save_dir_seg)
postprocess(save_dir_seg,save_dir_bin)

C:\Users\mrl\AppData\Local\Temp\ipykernel_3416\1223174814.py:30: UserWarning: C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\Mesmer_label_masks\6plexTonsil_DAPI_gray.tiff is a low contrast image
  io.imsave(os.path.join(save_dir_seg, file[:file.find('.')] + '.tiff'), label_mask)


C:\Users\mrl\AppData\Local\Temp\ipykernel_3416\1223174814.py:30: UserWarning: C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\Mesmer_label_masks\Bph2_45174_14597_DAPI_gray.tiff is a low contrast image
  io.imsave(os.path.join(save_dir_seg, file[:file.find('.')] + '.tiff'), label_mask)


C:\Users\mrl\AppData\Local\Temp\ipykernel_3416\1223174814.py:30: UserWarning: C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\Mesmer_label_masks\TME11_12154_44141_DAPI_gray.tiff is a low contrast image
  io.imsave(os.path.join(save_dir_seg, file[:file.find('.')] + '.tiff'), label_mask)


C:\Users\mrl\AppData\Local\Temp\ipykernel_3416\1223174814.py:30: UserWarning: C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\Mesmer_label_masks\Tonsil_7color_Scan2_17096_47011_gray.tiff is a low contrast image
  io.imsave(os.path.join(save_dir_seg, file[:file.find('.')] + '.tiff'), label_mask)


C:\Users\mrl\AppData\Local\Temp\ipykernel_3416\1223174814.py:30: UserWarning: C:\Users\mrl\Desktop\nuclear_seg_github\nuclear_segmentation\imgs\Mesmer_label_masks\VISTA4_47265_15281_DAPI_gray.tiff is a low contrast image
  io.imsave(os.path.join(save_dir_seg, file[:file.find('.')] + '.tiff'), label_mask)
